In [1]:
import cv2
import time
import torch
import argparse
import numpy as np
import csv
from torchvision import transforms
from utils.datasets import letterbox
from utils.torch_utils import select_device
from models.experimental import attempt_load
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from utils.general import non_max_suppression_kpt
from PIL import ImageFont, ImageDraw, Image

# Initialize the CSV file to store the data
csv_filename = 'fall_detection_output.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    headers = ['frame', 'xmin', 'ymin', 'xmax', 'ymax', 'fall_detected']
    for i in range(17):  # 17 keypoints
        headers.extend([f'kpt_{i}_x', f'kpt_{i}_y', f'kpt_{i}_conf'])
    writer.writerow(headers)

@torch.no_grad()
def run(poseweights='yolov7-w6-pose.pt', source='pose.mp4', device='cpu'):
    video_path = source
    device = select_device(device)
    model = attempt_load(poseweights, map_location=device)  # load FP32 model
    model.eval()
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print('Error while trying to read video. Please check path again')

    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    resize_factor = 1
    resized_width = int(original_width * resize_factor)
    resized_height = int(original_height * resize_factor)

    out_video_name = f"{video_path.split('/')[-1].split('.')[0]}"
    out = cv2.VideoWriter(f"{out_video_name}_test.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30,
                          (resized_width, resized_height))

    frame_count, total_fps = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (resized_width, resized_height))
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = letterbox(image, new_shape=resized_width, auto=False)[0]
            image = transforms.ToTensor()(image)
            image = torch.tensor(np.array([image.numpy()]))
            image = image.to(device).float()
            start_time = time.time()

            output, _ = model(image)
            output = non_max_suppression_kpt(output, 0.5, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'],
                                             kpt_label=True)
            output = output_to_keypoint(output)
            img = image[0].permute(1, 2, 0) * 255
            img = img.cpu().numpy().astype(np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            thre = (resized_height // 2) + 100
            for idx in range(output.shape[0]):
                kpts = output[idx, 7:].T
                plot_skeleton_kpts(img, kpts, 3)
                cx, cy, w, h = output[idx, 2], output[idx, 3], output[idx, 4], output[idx, 5]
                xmin, ymin = int(cx - w / 2), int(cy - h / 2)
                xmax, ymax = int(cx + w / 2), int(cy + h / 2)
                dx, dy = xmax - xmin, ymax - ymin
                
                # Ambil koordinat Y dari keypoint ke-2 (misalnya, pinggul)
                keypoint_index = 2
                ph = kpts[3 * keypoint_index + 1]  # Y-coordinate of keypoint 2
                
                difference = dy - dx
                fall_detected = ((difference < 0) and (ph > thre)) or (difference < 0)

                # Gambar bounding box
                cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                text = "FALL DETECTED" if fall_detected else "NORMAL"
                color = (0, 0, 255) if fall_detected else (255, 255, 255)1
                cv2.putText(img, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                
                # Simpan data ke CSV
                with open(csv_filename, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    row = [frame_count, xmin, ymin, xmax, ymax, fall_detected]
                    for i in range(0, len(kpts), 3):
                        kpt_x, kpt_y, kpt_conf = kpts[i], kpts[i+1], kpts[i+2]
                        row.extend([kpt_x, kpt_y, kpt_conf])
                    writer.writerow(row)
            
            cv2.imshow("Detection", img)
            cv2.waitKey(1)
            end_time = time.time()
            fps = 1 / (end_time - start_time)
            total_fps += fps
            frame_count += 1
            out.write(img)
        else:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    avg_fps = total_fps / frame_count
    print(f"Average FPS: {avg_fps:.3f}")

def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--poseweights', nargs='+', type=str, default='yolov7-w6-pose.pt', help='model path(s)')
    parser.add_argument('--source', type=str, default='0', help='source')  # file/folder, 0 for webcam
    parser.add_argument('--device', type=str, default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    opt = parser.parse_args()
    return opt

if __name__ == "__main__":
    opt = parse_opt()
    run(**vars(opt))


usage: ipykernel_launcher.py [-h] [--poseweights POSEWEIGHTS [POSEWEIGHTS ...]] [--source SOURCE] [--device DEVICE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\LENOVO\AppData\Roaming\jupyter\runtime\kernel-15f6fddb-f920-45d4-baa2-670819cc9b30.json


SystemExit: 2

C:\Users\LENOVO\anaconda3\envs\yolov7_env\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Fungsi untuk menghitung faktor panjang
def calculate_length_factor(shoulder, torso):
    return np.sqrt((shoulder[0] - torso[0])**2 + (shoulder[1] - torso[1])**2)

# Fungsi untuk mendeteksi jatuh
def detect_fall(keypoints, length_factor, alpha=0.1):
    left_shoulder = keypoints[5]  # Keypoint bahu kiri
    right_shoulder = keypoints[6]  # Keypoint bahu kanan
    left_hip = keypoints[11]  # Keypoint pinggang kiri
    right_hip = keypoints[12]  # Keypoint pinggang kanan
    left_foot = keypoints[15]  # Keypoint kaki kiri
    right_foot = keypoints[16]  # Keypoint kaki kanan

    # Periksa apakah bahu berada di bawah kaki
    shoulder_below_foot = left_shoulder[1] <= left_foot[1] + alpha * length_factor

    # Hitung tinggi dan lebar tubuh
    body_height = abs(left_shoulder[1] - left_foot[1])
    body_width = abs(left_shoulder[0] - right_shoulder[0])

    # Periksa apakah tinggi tubuh lebih kecil dari lebar tubuh
    height_less_than_width = body_height < body_width

    # Jika kedua kondisi terpenuhi, deteksi jatuh
    if shoulder_below_foot and height_less_than_width:
        return True
    return False

# Fungsi untuk memproses video dan menghitung metrik
def process_video(video_path, ground_truth_path):
    # Baca ground truth
    start_frame, end_frame, bbox_data = read_ground_truth(ground_truth_path)
    
    # Inisialisasi list untuk ground truth dan predictions
    ground_truth = []
    predictions = []
    
    # Buka video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Cek apakah frame termasuk dalam rentang jatuh
        if start_frame <= frame_count <= end_frame:
            ground_truth.append(1)  # 1 = jatuh
        else:
            ground_truth.append(0)  # 0 = tidak jatuh
        
        # Di sini Anda bisa menambahkan kode untuk menjalankan model YOLOv7-w6
        # dan mendapatkan keypoints untuk frame saat ini.
        # Contoh sederhana: prediksi jatuh jika frame berada dalam rentang tertentu.
        keypoints = get_keypoints_from_yolov7(frame)  # Ganti dengan fungsi yang sesuai
        
        # Hitung faktor panjang
        length_factor = calculate_length_factor(keypoints[5], keypoints[11])
        
        # Deteksi jatuh
        if detect_fall(keypoints, length_factor):
            predictions.append(1)  # Prediksi jatuh
        else:
            predictions.append(0)  # Prediksi tidak jatuh
        
        # Tampilkan frame (opsional)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Hitung metrik
    precision = precision_score(ground_truth, predictions)
    recall = recall_score(ground_truth, predictions)
    f1 = f1_score(ground_truth, predictions)
    
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")

# Path ke video dan file ground truth
video_path = "C:/Users/LENOVO/Documents/A Skripsi/datasets/FallDataset/Dataset/Coffee_room_01/Videos/video (1).avi"
ground_truth_path = "C:/Users/LENOVO/Documents/A Skripsi/datasets/FallDataset/Dataset/Coffee_room_01/Annotation_files/video (1).txt"

# Jalankan fungsi proses video
process_video(video_path, ground_truth_path)

NameError: name 'read_ground_truth' is not defined